In [1]:
%load_ext autoreload
%autoreload 2

from humbldata.core.utils.openbb_helpers import obb_login
from openbb import obb
from humbldata.core.utils.env import Env
import polars as pl

# obb_login()
obb.account.login(pat=Env().OBB_PAT, remember_me=True)

In [25]:
%load_ext autoreload
%autoreload

from humbldata.toolbox.toolbox_controller import Toolbox

toolbox = Toolbox(
    symbols="SPY,IWM,DIA,QQQ",
    interval="1d",
    start_date="2000-01-01",
    end_date="2024-12-03",
    # membership="anonymous"
)
toolbox

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Toolbox(symbols=['SPY', 'IWM', 'DIA', 'QQQ'], interval=1d, start_date=2023-12-04, end_date=2024-12-03, provider=yfinance, membership=anonymous, warnings=[{'category': 'ToolboxQueryParams', 'message': 'Start date adjusted to 2023-12-04 based on anonymous membership (1Y of data).'}])

In [26]:
# import nest_asyncio
# nest_asyncio.apply()

result = toolbox.technical.mandelbrot_channel(historical=False, window="1mo", _boundary_group_down=False, chart=True)
result

INFO: MandelbrotChannelFetcher || START: fetch_data (sync)
DEBUG: MandelbrotChannelFetcher || Running .transform_query()
DEBUG: MandelbrotChannelFetcher || Running .extract_data()
DEBUG: MandelbrotChannelFetcher || Running .transform_data()
INFO: MandelbrotChannelFetcher || END: fetch_data (sync) - Total time: 20.5835s


HumblObject

id: 0674f0fa-4af4-771f-8000-3fb2eae2c3ee
results: b"\xa1mDataFrameScan\xa2bdf\xa1gcolumns\x85\xa4dnameddatehdatatypedDatelbi...
equity_data: b'\xa1bIR\xa2gversion\x19%\x97cdsl\xa1mDataFrameScan\xa2bdf\xa1gcolumn...
provider: yfinance
warnings: [{'category': 'ToolboxQueryParams', 'message': 'Start date adjusted to 20...
chart: [{'content': '{"data":[{"line":{"color":"blue"},"name":"Recent Price","x":["...
extra: {}
context_params: {'symbols': ['SPY', 'IWM', 'DIA', 'QQQ'], 'interval': '1d', 'start_...
command_params: {'window': '1mo', 'rv_adjustment': True, 'rv_method': 'std', 'rs_me...

In [27]:
result.to_polars()

date,symbol,bottom_price,recent_price,top_price
date,str,f64,f64,f64
2024-12-02,"""DIA""",445.9471,448.73999,457.8957
2024-12-02,"""IWM""",240.1135,241.75,251.8686
2024-12-02,"""QQQ""",511.2826,515.289978,527.0281
2024-12-02,"""SPY""",601.9702,603.630005,614.5939


In [16]:
result.command_params

MandelbrotChannelQueryParams(window=1mo, rv_adjustment=True, rv_method=std, rs_method=RS, rv_grouped_mean=False, live_price=False, historical=False, chart=True, template=humbl_dark, _boundary_group_down=False)

In [17]:
result.context_params

Toolbox(symbols=['SPY', 'IWM', 'DIA', 'QQQ'], interval=1d, start_date=2023-12-04, end_date=2024-12-03, provider=yfinance, membership=anonymous, warnings=[{'category': 'ToolboxQueryParams', 'message': 'Start date adjusted to 2023-12-04 based on anonymous membership (1Y of data).'}])

## Test `calc_up_down_pct`


In [17]:
from humbldata.portfolio.analytics.user_table.helpers import calc_up_down_pct


df = pl.DataFrame({
    "symbol": ["AAPL", "GOOGL", "MSFT"],
    "bottom_price": [5.18, 15.07, 16.24],
    "recent_price": [10.05, 20.31, 16.42],
    "top_price": [11.23, 25.17, 30.09],
})
result = calc_up_down_pct(df)
print(result)

shape: (3, 6)
┌────────┬──────────────┬──────────────┬───────────┬─────────────────┬──────────┐
│ symbol ┆ bottom_price ┆ recent_price ┆ top_price ┆ ud_pct          ┆ ud_ratio │
│ ---    ┆ ---          ┆ ---          ┆ ---       ┆ ---             ┆ ---      │
│ str    ┆ f64          ┆ f64          ┆ f64       ┆ str             ┆ f64      │
╞════════╪══════════════╪══════════════╪═══════════╪═════════════════╪══════════╡
│ AAPL   ┆ 5.18         ┆ 10.05        ┆ 11.23     ┆ -48.46 / +11.74 ┆ 0.2      │
│ GOOGL  ┆ 15.07        ┆ 20.31        ┆ 25.17     ┆ -25.8 / +23.93  ┆ 0.48     │
│ MSFT   ┆ 16.24        ┆ 16.42        ┆ 30.09     ┆ -1.1 / +83.25   ┆ 0.99     │
└────────┴──────────────┴──────────────┴───────────┴─────────────────┴──────────┘


# Setting up UserTable

In [ ]:
%load_ext autoreload
%autoreload 2

from humbldata.core.standard_models.portfolio.analytics.user_table import UserTableQueryParams
from humbldata.core.utils.openbb_helpers import get_latest_price


symbols = UserTableQueryParams(symbol="AAPL, MSFT, NVDA").symbol
symbols

# Tesing Async


## Get Sector Async

In [ ]:
%load_ext autoreload
%autoreload 2

from humbldata.core.utils.openbb_helpers import aget_etf_category, aget_equity_sector
import polars as pl

stocks = pl.Series([
    "GOOGL",  # Communication Services
    "AMZN",   # Consumer Discretionary
    "KO",     # Consumer Staples
    "XOM",    # Energy
    "JPM",    # Financials
    "JNJ",    # Health Care
    "HON",    # Industrials
    "AAPL",   # Information Technology
    "LIN",    # Materials
    "PLD",    # Real Estate
    "NEE"     # Utilities
])

# Define objects and lists for different symbol groups

# Equities
equity_symbols = ["AAPL", "NVDA", "TSLA"]

# ETFs
etf_symbols = ["XLE", "XLF", "XLU"]

# Mixed (equities and ETFs)
mixed_symbols = ["AAPL", "XLE", "XLU", "TSLA"]

# Edge cases
edge_symbols = ["BITI", "LNGG", "ETHU", "DBA"]


# test = (await aget_etf_category(symbols=["AAPL", "XLE", "DBA"], provider="yfinance")).collect()
test2 = (await aget_equity_sector(symbols=equity_symbols, provider="yfinance")).collect()
# test2 = (await aget_equity_sector(symbols=["XLE", "AAPL"], provider="yfinance")).collect()
test2

In [ ]:
symbols = ["CORN", 'SLV', 'DBA', "XLE", "AAPL", "FXE", "SPY", "QQQ", "BITO", "MAXI", "SATO", "BIL", "LQD", "QAI", "MNA", "CLSE"]
symbols2 = ["AAPL", "FXE", "SPY", "MAXI", "GLD"]
symbols3 = commodity_etf_symbols = ["GLD", "IAU", "SLV", "GLDM", "PDBC", "SGOL", "FTGC", "DBC", "SIVR", "USO", "IAUM", "GSG", "BCI", "PPLT", "COMT", "GLTR", "OUNZ", "BAR", "UNG", "DBA", "AAAU", "DJP", "CMDT", "KRBN", "PALL", "COM", "NBCM", "CMDY", "DBO", "BCD", "KCCA", "CPER", "HGER", "USCI", "DBB", "DBP", "GCC", "WEAT", "BNO", "UGA", "DGP", "IGLD", "COMB", "DBE", "FGDL", "CORN", "USL", "CCRV", "DJCB", "IAUF", "PLTM", "BDRY", "PDBA", "BGLD", "UCIB", "SOYB", "GRN", "PIT", "BCIM", "RENW", "SDCI", "UNL", "SHNY", "DCMT", "TAGS", "CANE", "TMET", "KEUA", "HCOM", "HARD", "EVMT", "USG", "AMPD", "DZZ", "DGZ", "KMET", "CMCI", "USOY", "ZSC", "TILL", "BWET", "DULL", "ZSB", "LNGG", "USOI", "OILK", "SLVO", "GLDI", "BOIL", "KOLD", "AGQ", "GLL", "UGL", "ZSL", "UCO", "SCO"]



df = (await aget_etf_category(symbols=edge_symbols)).collect()
df

In [ ]:
from humbldata.portfolio.analytics.user_table.helpers import normalize_asset_class


normalized_df = normalize_asset_class(df).select(["symbol", "category"])
normalized_df

In [ ]:
%load_ext autoreload
%autoreload 2

from humbldata.core.utils.openbb_helpers import aget_etf_category


df2 = (await aget_etf_category(symbols=edge_symbols)).collect()
df2

In [ ]:
%load_ext autoreload
%autoreload 2

from humbldata.portfolio.analytics.user_table.helpers import aget_asset_class_filter

(await aget_asset_class_filter(symbols=["XLE", "XLF", "XLK"])).collect()

## Test aget_sector_filter

In [ ]:
%load_ext autoreload
%autoreload 2

from humbldata.portfolio.analytics.user_table.helpers import aget_sector_filter

etf_data = pl.LazyFrame({
    "symbol": ["BITI", "LNGG", "ETHU", "DBA"],
    "category": [
        "Trading--Miscellaneous",
        "Equity Energy",
        None,
        "Commodities Focused",
    ]
})

(await aget_sector_filter(symbols=mixed_symbols)).collect()

# Test user_table_engine

In [ ]:
from humbldata.portfolio.analytics.user_table.model import user_table_engine
import polars as pl

etf_data = [
    pl.DataFrame(
        {"symbol": ["AAPL", "NVDA", "TSLA"], "category": [None, None, None]}
    ),
    pl.DataFrame(
        {
            "symbol": ["XLE", "XLF", "XLU"],
            "category": ["Equity Energy", "Financial", "Utilities"],
        }
    ),
    pl.DataFrame(
        {
            "symbol": ["AAPL", "XLE", "XLU", "TSLA"],
            "category": [None, "Equity Energy", "Utilities", None],
        }
    ),
    pl.DataFrame(
        {
            "symbol": ["BITI", "LNGG", "ETHU", "DBA"],
            "category": [
                "Trading--Miscellaneous",
                "Equity Energy",
                None,
                "Commodities Focused",
            ],
        }
    ),
]

out: pl.LazyFrame = await user_table_engine(symbols=edge_symbols)
out.collect()

## Test UserTableFetcher

In [ ]:
%load_ext autoreload
%autoreload 2

from humbldata.portfolio.portfolio_controller import Portfolio

portfolio = Portfolio(symbols=["AAPL"], user_role="anonymous")
portfolio

In [ ]:
result = (await portfolio.analytics.user_table())

In [ ]:
result.to_polars()

In [ ]:
result.to_polars()

# Testing Sandbox

In [ ]:
%load_ext autoreload
%autoreload 2

from humbldata.toolbox.toolbox_controller import Toolbox
import datetime
from datetime import timedelta

# start_date = "2020-01-01"
# end_date = "2024-01-01"
start_date = (datetime.datetime.now() - datetime.timedelta(days=365))
end_date = datetime.date(2024, 7, 18)

toolbox = Toolbox(
    symbols=["AAPL", "AMD"],
    interval="1d",
    start_date="2022-07-18",
    end_date=end_date,
    membership="peon"
)

In [ ]:
result = toolbox.technical.mandelbrot_channel(historical=False, window="1m", _boundary_group_down=False, chart=True)
result


In [ ]:
result.warnings[0]

In [ ]:
import pickle
from pathlib import Path

# Ensure the directory exists
Path("../tests/test_data").mkdir(parents=True, exist_ok=True)

# Save the result object to the specified file
with Path("../tests/test_data/mandelbrot_humblobject.pkl").open("wb") as file:
    pickle.dump(result, file)

print("Result object saved successfully.")


In [ ]:
import pickle

with open("../tests/test_data/mandelbrot_humblobject.pkl", "rb") as file:
    out = pickle.load(file)
    file.close()

In [10]:
from humbldata.core.utils.openbb_helpers import aget_last_close


(await aget_last_close(symbols=["AAPL, XLE"])).collect().to_dicts()

[{'symbol': 'AAPL', 'prev_close': 218.24}]

In [8]:
symbols="AAPL,XLE"
symbols.split(",")

['AAPL', 'XLE']